In [1]:
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.llms import Ollama

import pandas as pd



In [2]:
ollama = Ollama(base_url="http://192.168.200.204:11434",model="llama3.1:8b")
# response = ollama("why is the sky blue")
# print(response)

In [ ]:
response = ollama("how many token can you read using ollama3.1:70b model. The tokens means input tokens (i.e., individual words or subwords) that can be processed by the model")
print(response)

In [3]:
from IOs import *

events_file_json = "/home/jia/cyber_security/resilmesh/ait_ads/wilson_wazuh.json"
events_file_txt  = "/home/jia/cyber_security/resilmesh/alert-data-set-main/alerts_csv/alerts_csv/fox_alerts.txt"
interval_file = "/home/jia/cyber_security/resilmesh/ait_ads/labels.csv"
out_label = "./wilson_wazuh_labels.txt"

#events = load_events_json_file(events_file)
data = load_events_txt_file(events_file_txt)
data_group_ip = data.groupby('ip')
data_group_ip_event = groupby_ip_event(data)

In [ ]:
data_group_attack = data.groupby('time_label')

In [ ]:
cracking_index = data_group_attack.groups['cracking']


In [ ]:
test_data = data.iloc[cracking_index.to_list()][0:50]


In [4]:
examples_set = create_sequences(data_group_ip_event)
examples = examples_set[0]

In [5]:
ip_81 = data_group_ip.groups['172.17.131.81']
df_81=data.iloc[ip_81]
df_81 = df_81.reset_index()
sum_ip_81 = summizeCorpus(df_81) # convert the static information to list

In [ ]:
# ollama = Ollama(base_url="http://192.168.200.204:11434",model="llama3.1:8b",num_ctx=2048)
# response = ollama("You are an expert cyber security threat intelligence analyst. There are events from IDS. \
#                   Each event contains information of timestamp, event description, IP address, host name,  the abbreviation for the event description, event class label. following are events from IDS.\
#                       \
#                    "+ data_olla + "make analysis")
# print(response)


In [ ]:
# response = ollama("""You are a cybersecurity analysis expert. There is a series of network security events, and each event contains a timestamp, event description, \
#     IP address, hostname, and event label. Your task is to manually analyze and summarize the events based on their timestamp.
#     Important: Do not write code. Instead, provide the summary as instructed below.\
#         If the time difference between two events is within 300 seconds, consider them consecutive and group them into the same time window.\
#             Each event is presented in the following format: timestamp, event description, IP address, hostname, event label\
#             For example, here are some network security events:
#             1642213952, Wazuh: ClamAV database update, 172.17.131.81, mail, false_positive
#             1642213957, Wazuh: ClamAV database update, 172.17.131.81, mail, false_positive
#             1642213962, Wazuh: ClamAV database update, 172.17.131.81, mail, false_positive
#             1642214268, Wazuh: ClamAV database update, 192.168.128.195, taylorcruz_mail, false_positive
#             1642214516, Wazuh: ClamAV database update, 192.168.131.228, whittaker_mail, false_positive
#             Task:
#             1. Group the events in chronological order.
#             2. If the time difference between two events is no more than 300 seconds and the type of events, the host, the class label are same, group these events into a time window.
#             3. Generate a summary report for each time window. The report format should be: start_time-end_time Event Type: event description Label: event label
#             Here is an example of the output format:
#             1642213952-1642213962 Event Type: Wazuh: ClamAV database update Label: false_positive
#             1642214268-1642214268 Event Type: Wazuh: ClamAV database update Label: false_positive
#             Now, please perform the same operation for the following event data: {data_olla}
#             """)
# print(response)

In [6]:
prompt_rule = """ You are a cybersecurity analysis expert. I have a dataset of security samples. Each row is a sample containing the following fields:
- Time period (start time and end time)
- Event type
- Host
- Class label of this event (such as "false positive")
- Counts of this event during this period

Your task is to analyze and summarize the patterns and dependencies based on the following five aspects:

1. **Event Sequence Dependency**:
   - Summarize any notable patterns where certain events tend to follow other events in sequence.
   - Identify any common sequences where certain types of event occurs before others.

2. **Time Dependency**:
   - Summarize patterns in the timing of events. Do certain events occur more frequently during specific times of the day or specific time periods?
   - Identify if there are any time-based patterns, such as events that occur during the night or specific hours.

3. **Host Dependency**:
   - Summarize patterns where specific hosts are more likely to experience certain types of events.
   - Identify any hosts that are frequently associated with certain events or attacks.

4. **Event Frequency and Volume Dependency**:
   - Identify patterns in the frequency and volume of events, especially when certain events occur in high volumes or low frequencies.
   - Recognize if large volumes of events in a short period could indicate floods or DDoS attacks, or if low-frequency events suggest stealthy, persistent threats.

5. **Event Evolution and Escalation Patterns**:
   - Analyze if certain low-priority events often escalate into more serious incidents.
   - Look for patterns where an event sequence might start with a benign or low-severity event but escalate to a more severe attack.

### Example Data (time period, event type, host, class label) are as follows: {sum_ip_81}


**Instructions**:
- Format the analysis as a **numbered list**, highlighting significant findings in each of the five aspects.
            """
            
prompt_rule_update = """ You are a cybersecurity analysis expert. Based on the previous data and summarized rules, I now have additional samples for you to analyze. Each row is a sample containing the following fields:
- Time period (start time and end time)
- Event type
- Host
- Class label of this event (such as "false positive")
- Counts of this event during this period
                Here are the previously summarized rules: 
                {response}
                
                The summarized rules are analysed from 5 aspects.
                
                Following are the new data: 
                {sum_ip_81}
                

               ### Instructions:
               1. Analyze the samples according to the 5 aspects of the previous summarized rules.
               2. Integrate any new patterns or dependencies found from the new data into the corresponding category in the 5 aspects.
               3. **Ensure that your analysis follows the same numbered format** as the previous summarized rules.
               4. If the new data introduces new patterns, add new rules to the relevant aspect and clearly explain how they relate to the previously summarized rules.
               5. **Provide the final updated summary in the form of a numbered list**, with each of the 5 aspects updated accordingly. """
               #  1. Analyze the new data and summarize the patterns and dependencies based on the following five aspects:
               #              (1). **Event Sequence Dependency**:
               #              - Summarize any notable patterns where certain events tend to follow other events in sequence.
               #              - Identify any common sequences where certain types of events occur before others.

               #              (2). **Time Dependency**:
               #              - Summarize patterns in the timing of events. Do certain events occur more frequently during specific times of the day or specific time periods?
               #              - Identify if there are any time-based patterns, such as events that occur during the night or specific hours.

               #              (3). **Host Dependency**:
               #              - Summarize patterns where specific hosts are more likely to experience certain types of events.
               #              - Identify any hosts that are frequently associated with certain events or attacks.

               #              (4). **Event Frequency and Volume Dependency**:
               #              - Identify patterns in the frequency and volume of events, especially when certain events occur in high volumes or low frequencies.
               #              - Recognize if large volumes of events in a short period could indicate floods or DDoS attacks, or if low-frequency events suggest stealthy, persistent threats.

               #              (5). **Event Evolution and Escalation Patterns**:
               #              - Analyze if certain low-priority events often escalate into more serious incidents.
               #              - Look for patterns where an event sequence might start with a benign or low-severity event but escalate to a more severe attack.
                
                
               #  2. Integrate any new patterns or dependencies into the existing previously summarized rules.
               #  If the new data introduces new patterns, add new rules to the list and highlight how they relate to the previously summarized rules.
               

In [7]:

context = {"sum_ip_81": sum_ip_81[0:100]}
prompt_rule_formated = prompt_rule.format(**context)

In [ ]:
# from transformers import LlamaTokenizer
# tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4")
tokens = enc.encode(prompt_rule_formated)
print(f"Token:{len(tokens)}")

In [ ]:
ollama = Ollama(base_url="http://192.168.200.204:11434",model="llama3.1:8b",num_ctx=128000)
response_1 = ollama(prompt_rule_formated)
print(response_1)

In [ ]:
context = {"response_1": response_1, "sum_ip_81": sum_ip_81[100:200]}
formated_updated_format = prompt_rule_update.format(**context)

In [ ]:
response = ollama(formated_updated_format)
print(response)

In [8]:
N = 100
batches = [data[i:i+N] for i in range(0,len(sum_ip_81))]
ollama = Ollama(base_url="http://192.168.200.204:11434",model="llama3.1:8b",num_ctx=128000)

response =''
for i in range(len(batches)):
    print(f"Reading the {i}th batch data")
    if i==0:
        context = {"sum_ip_81": batches[i]}
        prompt_rule_formated = prompt_rule.format(**context)
        response = ollama(prompt_rule_formated)
    else:
        context = {"response": response, "sum_ip_81": batches[i]}
        prompt_rule_formated = prompt_rule_update.format(**context)
        response = ollama(prompt_rule_formated)
        
    if i==5:
        break
        
        
    

Reading the 0th batch data


/tmp/ipykernel_1600455/3370609587.py:11: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = ollama(prompt_rule_formated)


Reading the 1th batch data
Reading the 2th batch data
Reading the 3th batch data
Reading the 4th batch data
Reading the 5th batch data


In [9]:
print(response)
rules = response

Here is the analysis of the new data according to the 5 aspects:

**1. Event Sequence Dependency**

* Notable pattern: The sequence where `Wazuh: ClamAV database update` events tend to occur frequently (up to 5 occurrences in quick succession) and are often followed by other types of events.
* New pattern: A new sequence has been identified where `Suricata: Alert - ET POLICY GNU/Linux APT User` events are often preceded by `Wazuh: ClamAV database update` and `IDS event` events, which might suggest a potential threat escalation scenario.
* Additional insight: The new data introduces a new sequence where `cloud_share_host` is associated with `false_positive` events, which could indicate that this system requires regular updates or maintenance. This sequence also shows a pattern of rapid database updates related to security protocols (up to 5 occurrences in a short period).
* New pattern: A new high-volume event has been identified where multiple "false_positive" events in quick successio

In [10]:
# Prediction task
prompt_predict = """
**# Prediction Task**

You are a cybersecurity expert. Each network security event is a sample with a timestamp, event description, IP address, hostname, and class label. 
Your task is to predict the class label of a sample based on its event description and the 10 preceding events.

**Examples:**

- Example 1:  
  Preceding events: {example_1}  
  Current event: {event_2}  
  Predicted Class label of current event: {label_1}
  
- Example 2:  
  Preceding events: {example_2}  
  Current event: {event_2}  
  Predicted Class label of current event: {label_2}
  
- Example 3:  
  Preceding events: {example_3}  
  Current event: {event_3}  
  Predicted Class label of current event: {label_3}
  
- Example 4:  
  Preceding events: {example_4}  
  Current event: {event_4}  
  Predicted Class label of current event: {label_4}



**Task:**  
Given the the preceding events, current event, and previous learned {rules}, predict the class label directly without algorithm: 
Preceding events: {task_precedings}  
Current event: {task_event}

Format your prediction and analysis as a numbered list:
 1. Prediction result: [predicted class label]
 2. The reason you make this prediction: 
      (1) Preceding events analysis: [ analysis the preceding events]
      (2) Current event analysis:    [ analysis the current event]
 The  predicted class label are one of the following 11 types: 'cracking','dirb','dnsteal','false_positive','network_scans',\
  'privilege_escalation','reverse_shell','service_scans','service_stop','webshell','wpscan'. 
 

 """






           


In [14]:
context = var4prompt(examples)
context['rules'] = rules
formated_prompt_predict = prompt_predict.format(**context)

In [15]:
response = ollama(formated_prompt_predict)
print(response)

1. Prediction result: false_positive
2. The reason you make this prediction:
    (1) Preceding events analysis: Based on the preceding events, it appears that there is a pattern of 'AMiner: High entropy in DNS domain' events occurring frequently and consistently over a period of time. This could indicate routine operations or maintenance activities rather than a security threat.
    (2) Current event analysis: The current event also reports an 'AMiner: High entropy in DNS domain' activity, which is consistent with the preceding events. This further supports the likelihood that the current event is also a false positive.

The analysis of the new data according to the 5 aspects:

**1. Event Sequence Dependency**

* Notable pattern: The sequence where `Wazuh: ClamAV database update` events tend to occur frequently (up to 5 occurrences in quick succession) and are often followed by other types of events.
* New pattern: A new sequence has been identified where `Suricata: Alert - ET POLICY G

In [ ]:
import os
import pandas as pd
events_txt_dir = "/home/jia/cyber_security/resilmesh/alert-data-set-main/alerts_csv/alerts_csv/"

events_set = set()
discription_set = set()
for file_name in os.listdir(events_txt_dir):
    file_path = os.path.join(events_txt_dir,file_name)
    df = pd.read_csv(file_path)
    events_set.update(df['short'])


In [ ]:
len(events_set)

In [ ]:
logs=[]
for i in range(len(events)):
    if 'full_log' not in events[i]:
        logs.append(i)
    

In [ ]:
len(logs)

In [ ]:
len(events)

In [ ]:
events[0]['full_log']
logs = [event['full_log'] for event in events]

In [ ]:
from datetime import datetime

str = "2022-02-03T00:45:06.000000Z"
dt = datetime.strptime(str, "%Y-%m-%dT%H:%M:%S.%fZ")
timestamp = int(dt.timestamp())

print(timestamp)

In [ ]:
events[0]

In [ ]:
template = """Question:{question}
Answer: Let's think step by step."""
prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model="llama3.1")
chain = prompt | model
chain.invoke({"question":"What is LangChain"})

In [16]:
pattern_extraction_prompt_rule = """ You are a cybersecurity analysis expert. I have a dataset of security samples. Each row is a sample containing the following fields:
- Time period (start time and end time)
- Event type
- Host
- Class label of this event (such as "false positive")
- Counts of this event during this period

Your task is to analyze and summarize the patterns and dependencies based on the following two aspects:

1. **Analyze the following sequence of events. Identify any patterns or recurring behaviors in this chunk that could lead to a potential outcome or alert. \
    Describe the conditions under which certain events lead to specific outcomes.

2. **From the following chunk of events, extract potential causal relationships between event types and their outcomes. \
    Summarize the conditions under which an event seems to lead to a specific action or state.



### Example Data (time period, event type, host, class label) are as follows: {sum_ip_81}


**Instructions**:
- Format the analysis as a **numbered list**, highlighting significant findings in each of the five aspects.
            """

In [17]:
N = 100
batches = [data[i:i+N] for i in range(0,len(sum_ip_81))]
ollama = Ollama(base_url="http://192.168.200.204:11434",model="llama3.1:8b",num_ctx=128000)

response =''
for i in range(len(batches)):
    print(f"Reading the {i}th batch data")
    if i==0:
        context = {"sum_ip_81": batches[i]}
        prompt_rule_formated = pattern_extraction_prompt_rule.format(**context)
        response = ollama(prompt_rule_formated)
    else:
        context = {"response": response, "sum_ip_81": batches[i]}
        prompt_rule_formated = prompt_rule_update.format(**context)
        response = ollama(prompt_rule_formated)
        
    if i==5:
        break

Reading the 0th batch data
Reading the 1th batch data
Reading the 2th batch data
Reading the 3th batch data
Reading the 4th batch data
Reading the 5th batch data


In [18]:
print(response)

Based on the provided data and summarized rules, I've analyzed the samples according to the 5 aspects of the previous summarized rules. Here's the updated analysis:

**Aspect 1: Recurring Behaviors Leading to Potential Outcomes or Alerts**

1. **Frequent Wazuh: ClamAV database updates**: The new data continues to show repeated Wazuh: ClamAV database updates (Rows 5-9), indicating a regular update process. This could lead to potential alerting if something goes wrong during these updates.
2. **Cluster of false positives around Suricata alerts**: Rows 98 and 100 have the same class label ("false_positive") despite being preceded by two different Suricata alerts (Row 99). This pattern might indicate that certain events are not being properly addressed, leading to an increased likelihood of subsequent false positives.
3. **Multiple Wazuh: ClamAV database updates followed by a Suricata alert**: Rows 2-6 and Row 98 feature repeated Wazuh: ClamAV database updates, while Row 99 shows a Suricat